In [1]:
#!/usr/bin/env python
# coding: utf-8
# In[1]:
#!/usr/bin/env python
# coding: utf-8
# get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
import glob
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 
from sklearn.model_selection import StratifiedKFold, KFold
import gc
from tqdm import tqdm_notebook, tqdm
import json
from typing import NamedTuple
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
print(torch.__version__)
# from tools import eval_summary, save_feature_importance, merge_preds

device = torch.device('cpu')
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

/home/aiden/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


2.3.2
1.3.1
GeForce RTX 2070 SUPER


device(type='cuda', index=0)

In [2]:
# In[2]:
torch.set_num_threads(8)
torch.get_num_threads()

8

#### Load Data

In [3]:
df_train = pd.read_csv('input/train.csv', dtype=np.float32)
df_test = pd.read_csv('input/test.csv', dtype=np.float32)
print(df_train.shape, df_test.shape)


(810000, 230) (10000, 227)


In [4]:
df1 = pd.read_csv('input/SiO2.txt', sep='\t')
df2 = pd.read_csv('input/Si3N4.txt', sep='\t')

df_nk = pd.merge(df1, df2, on='Wavelength(nm)')
df_nk = df_nk[:226]

In [5]:
df_nk.columns

Index(['Wavelength(nm)', 'n_x', 'k_x', 'n_y', 'k_y'], dtype='object')

In [46]:
df_nk[['n_x']].T.values

array([[1.5384, 1.5285, 1.5202, 1.5133, 1.5074, 1.5024, 1.498 , 1.4942,
        1.4908, 1.4878, 1.4851, 1.4827, 1.4806, 1.4787, 1.4769, 1.4753,
        1.4738, 1.4725, 1.4713, 1.4701, 1.4691, 1.4681, 1.4672, 1.4663,
        1.4656, 1.4648, 1.4641, 1.4635, 1.4629, 1.4623, 1.4618, 1.4613,
        1.4608, 1.4603, 1.4599, 1.4595, 1.4591, 1.4587, 1.4584, 1.458 ,
        1.4577, 1.4574, 1.4571, 1.4568, 1.4565, 1.4563, 1.456 , 1.4558,
        1.4555, 1.4553, 1.4551, 1.4549, 1.4546, 1.4544, 1.4542, 1.454 ,
        1.4539, 1.4537, 1.4535, 1.4533, 1.4531, 1.453 , 1.4528, 1.4527,
        1.4525, 1.4523, 1.4522, 1.452 , 1.4519, 1.4518, 1.4516, 1.4515,
        1.4513, 1.4512, 1.4511, 1.4509, 1.4508, 1.4507, 1.4505, 1.4504,
        1.4503, 1.4502, 1.45  , 1.4499, 1.4498, 1.4497, 1.4496, 1.4494,
        1.4493, 1.4492, 1.4491, 1.449 , 1.4489, 1.4487, 1.4486, 1.4485,
        1.4484, 1.4483, 1.4482, 1.4481, 1.4479, 1.4478, 1.4477, 1.4476,
        1.4475, 1.4474, 1.4473, 1.4471, 1.447 , 1.4469, 1.4468, 

In [6]:
layer_cols = [c for c in df_train.columns if 'layer_' in c]
fea_cols = [c for c in df_train.columns if c not in layer_cols]

len(fea_cols), len(layer_cols)

(226, 4)

In [7]:
df_model = df_train

In [8]:
# size = 48
W = 15 # input_volume_size
F = 6  # kernel_size
S = 1   # strides
P = 1
# padding_size

size = (W - F + 2*P) / S + 1
size
# ((size - 1) * S) - 2*P + F

12.0

#### Model

In [47]:
class DNN1Model(torch.nn.Module):
    def __init__(self, input_size, dropout_probability=0.3):
        super().__init__()
        relu = torch.nn.ReLU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.layer_1 = torch.nn.Sequential(
            torch.nn.Linear(input_size, input_size), relu, torch.nn.BatchNorm1d(input_size), dropout, 
            torch.nn.Linear(input_size, input_size), relu, torch.nn.BatchNorm1d(input_size), dropout, 
            torch.nn.Linear(input_size, input_size), relu, torch.nn.BatchNorm1d(input_size), dropout, 
        )
        self.layer_2 = torch.nn.Sequential(
            torch.nn.Linear(input_size, input_size), relu, torch.nn.BatchNorm1d(input_size), dropout, 
            torch.nn.Linear(input_size, input_size), relu, torch.nn.BatchNorm1d(input_size), dropout, 
            torch.nn.Linear(input_size, input_size), relu, torch.nn.BatchNorm1d(input_size), dropout, 
        )
        self.layer_3 = torch.nn.Sequential(
            torch.nn.Linear(input_size, input_size), relu, torch.nn.BatchNorm1d(input_size), dropout, 
            torch.nn.Linear(input_size, input_size), relu, torch.nn.BatchNorm1d(input_size), dropout, 
            torch.nn.Linear(input_size, input_size), relu, torch.nn.BatchNorm1d(input_size), dropout, 
        )
        self.layer_4 = torch.nn.Sequential(
            torch.nn.Linear(input_size, input_size), relu, torch.nn.BatchNorm1d(input_size), dropout, 
            torch.nn.Linear(input_size, input_size), relu, torch.nn.BatchNorm1d(input_size), dropout, 
            torch.nn.Linear(input_size, input_size), relu, torch.nn.BatchNorm1d(input_size), dropout, 
        )
        
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(input_size, input_size), relu,
            torch.nn.Linear(input_size, input_size), relu,
            torch.nn.Linear(input_size, input_size), relu,
            torch.nn.Linear(input_size, 4),
        )
        
        self.layer13_n = torch.Tensor(df_nk[['n_x']].T.values).to(device)
        self.layer13_k = torch.Tensor(df_nk[['k_x']].T.values).to(device)
        self.layer24_n = torch.Tensor(df_nk[['n_y']].T.values).to(device)
        self.layer24_k = torch.Tensor(df_nk[['k_y']].T.values).to(device)
        
    
    def forward(self, x_fea):
        
        out_layer_1 = self.layer_1(torch.add(torch.mul(x_fea, self.layer13_n), self.layer13_k))
        out_layer_2 = self.layer_2(torch.add(torch.mul(out_layer_1, self.layer24_n), self.layer24_k))
        out_layer_3 = self.layer_3(torch.add(torch.mul(out_layer_2, self.layer13_n), self.layer13_k))
        out_layer_4 = self.layer_4(torch.add(torch.mul(out_layer_2, self.layer24_n), self.layer24_k))
        
        return self.fc(out_layer_4)
    

In [48]:
class CNNModel(torch.nn.Module):
    def __init__(self, dropout_probability=0.3):
        super().__init__()
        relu = torch.nn.ReLU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.cnn = torch.nn.Sequential(
            torch.nn.Conv1d(1, 2, 31, stride=1, padding=0), #196
            relu, torch.nn.MaxPool1d(2), #98 
            torch.nn.Conv1d(2, 4, 19, stride=1, padding=0), #80
            relu, torch.nn.MaxPool1d(2), #40
            torch.nn.Conv1d(4, 8, 11, stride=1, padding=0), #30
            relu, torch.nn.MaxPool1d(2), #15
            torch.nn.Conv1d(8, 16, 6, stride=1, padding=1), #12
            relu, torch.nn.MaxPool1d(2), #6
        )
            
# #             torch.nn.Linear(input_size, 4),
#             torch.nn.Linear(input_size, 200), relu, #torch.nn.BatchNorm1d(200), dropout, 
#             torch.nn.Linear(200, 200), relu, #torch.nn.BatchNorm1d(200), dropout,
#             torch.nn.Linear(200, 200), relu, #torch.nn.BatchNorm1d(200), dropout,
#             torch.nn.Linear(200, 150), relu, #torch.nn.BatchNorm1d(200), dropout,
#             torch.nn.Linear(150, 128), relu, #torch.nn.BatchNorm1d(128), dropout,
#             torch.nn.Linear(128, 128), relu, #torch.nn.BatchNorm1d(128), dropout,
#             torch.nn.Linear(128, 100), relu, #torch.nn.BatchNorm1d(128), dropout,
#             torch.nn.Linear(100, 64), relu, #torch.nn.BatchNorm1d(128), dropout,
#             torch.nn.Linear(64, 32), relu, #torch.nn.BatchNorm1d(128), dropout,
#             torch.nn.Linear(32, 16), relu, #torch.nn.BatchNorm1d(128), dropout,
#             torch.nn.Linear(16, 8), relu, #torch.nn.BatchNorm1d(128), dropout,
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(16*6, 64), relu, #torch.nn.BatchNorm1d(128), dropout,
            torch.nn.Linear(64, 4)
       )
    def forward(self, x):
        x = x.unsqueeze(1)
        out = self.cnn(x)
        dim = 1
        for d in out.size()[1:]: #24, 4, 4
            dim = dim * d
        out = out.view(-1, dim)
        out = self.fc(out)
        return out

        return self.model(x)
    
class DNNModel(torch.nn.Module):
    def __init__(self, input_size, dropout_probability=0.3):
        super(DNNModel,self).__init__()
        relu = torch.nn.ReLU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.model = torch.nn.Sequential(
#             torch.nn.Linear(input_size, 4),
#             torch.nn.Linear(input_size, 1),
            torch.nn.Linear(input_size, 200), relu, torch.nn.BatchNorm1d(200), dropout, 
            torch.nn.Linear(200, 150), relu, torch.nn.BatchNorm1d(150), dropout,
            torch.nn.Linear(150, 100), relu, torch.nn.BatchNorm1d(100), dropout,            
            torch.nn.Linear(100, 64), relu, torch.nn.BatchNorm1d(64), dropout,
            torch.nn.Linear(64, 32), relu, torch.nn.BatchNorm1d(32), dropout,            
            torch.nn.Linear(32, 4)
                           )
    def forward(self, x):
        return self.model(x)
    
class SemiDataset(Dataset):
    def __init__(self, df, fea_cols, y_cols):        
        self.X = df[fea_cols].values
        self.y = df[y_cols].values
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx].astype(np.float32), self.y[idx].astype(np.float32)
    

#### Trainer

In [49]:
class Trainer(object):
    def __init__(self, model, criterion, optimizer, scheduler, device):
        self.device = device
        self.model = model#.to(self.device)
        self.criterion = criterion#.to(self.device)
        self.optimizer = optimizer
        self.scheduler = scheduler

        print(self.model.train())
        pass
    
    def train(self, data_loader):
        self.model.train()
        total_loss = 0
#         print('train_loader', len(train_loader))
        for data in data_loader:
            X_batch, y_batch = data
            X_batch = X_batch.to(self.device)
            y_batch = y_batch.to(self.device)
            
            y_pred = self.model(X_batch)
#             print(y_pred, y_batch)
            
            loss = self.criterion(y_pred, y_batch)
            total_loss = total_loss + loss.item()

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        
#         print(f'total_loss {total_loss / len(data_loader)}')
        return total_loss / len(data_loader)
    
    def eval(self, data_loader):
        self.model.eval()
        total_loss = 0
#         print('valid_loader', len(valid_loader))
        for data in data_loader:
            X_batch, y_batch = data
            X_batch = X_batch.to(self.device)
            y_batch = y_batch.to(self.device)
            with torch.no_grad():
                y_pred = self.model(X_batch)
                loss = self.criterion(y_pred, y_batch)
                print(y_pred, y_batch)
#                 print(loss.item())
                total_loss = total_loss + loss.item()
#         print(f'total_loss {total_loss / len(data_loader)}')
        return total_loss / len(data_loader)

    def save(self, model_path='checkpoint.pt'):
        torch.save(self.model.state_dict(), 'checkpoint.pt')
        return
    
    def load(self, model_path='checkpoint.pt'):
        self.model.load_state_dict(torch.load(model_path))
        return

#### Train

In [50]:
model_ts = datetime.now().strftime('%Y%m%dT%H%M%S')
print(model_ts)

batch_size = 10000
num_workers = 0

print(f'batch_size {batch_size} num_workers {num_workers}')
print(f'fea_size {len(fea_cols)} layer_cols {layer_cols}')


model = DNN1Model(input_size=len(fea_cols), dropout_probability=0.5).to(device)
#     model = CNNModel(dropout_probability=0.1).to(device)

    
criterion = nn.L1Loss(reduction='mean').to(device)
# criterion = nn.MSELoss(reduction='mean').to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
#     optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

scheduler = StepLR(optimizer, step_size=400, gamma=0.97)

trainer = Trainer(model, criterion, optimizer, scheduler, device)
# trainer.load()

train_dataset = SemiDataset(df_model[fea_cols + layer_cols], fea_cols, layer_cols)

train_loader_params = {
    'dataset' : train_dataset,
    'batch_size' : batch_size,
    'shuffle' : False,
    'num_workers' : num_workers,
    'drop_last' : False,
}
train_loader = DataLoader(**train_loader_params)

print(f'train_loader {len(train_loader)}')

20200116T190052
batch_size 10000 num_workers 0
fea_size 226 layer_cols ['layer_1', 'layer_2', 'layer_3', 'layer_4']
DNN1Model(
  (layer_1): Sequential(
    (0): Linear(in_features=226, out_features=226, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(226, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=226, out_features=226, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(226, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=226, out_features=226, bias=True)
    (9): ReLU()
    (10): BatchNorm1d(226, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.5, inplace=False)
  )
  (layer_2): Sequential(
    (0): Linear(in_features=226, out_features=226, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(226, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.5, inpl

In [51]:
# model.load_state_dict(torch.load('checkpoint.pt'))

In [ ]:
total_epoch = 1000

for e in tqdm_notebook(range(total_epoch), total=total_epoch, desc='Epoch'):
    loss = trainer.train(train_loader)
    
    ts = datetime.now().strftime('%Y%m%dT%H%M%S')
    print(f'[{ts}] Epock {e} / {total_epoch} loss: {loss}')
    
    torch.save(model.state_dict(), 'checkpoint.pt')

[20200116T190101] Epock 0 / 1000 loss: 78.78073454491886
[20200116T190111] Epock 1 / 1000 loss: 70.00115914992344
[20200116T190120] Epock 2 / 1000 loss: 69.00374363086841
[20200116T190129] Epock 3 / 1000 loss: 67.31920623779297
[20200116T190138] Epock 4 / 1000 loss: 62.18537177568601
[20200116T190148] Epock 5 / 1000 loss: 61.07716991283275
[20200116T190157] Epock 6 / 1000 loss: 61.415932031325355
[20200116T190206] Epock 7 / 1000 loss: 61.62692025267047
[20200116T190215] Epock 8 / 1000 loss: 61.63267540637358
[20200116T190224] Epock 9 / 1000 loss: 61.54740500744478
[20200116T190233] Epock 10 / 1000 loss: 61.8875867113655
[20200116T190242] Epock 11 / 1000 loss: 61.84366113168222
[20200116T190251] Epock 12 / 1000 loss: 62.20671599588277
[20200116T190301] Epock 13 / 1000 loss: 62.16505973721728
[20200116T190310] Epock 14 / 1000 loss: 61.37394827383536
[20200116T190319] Epock 15 / 1000 loss: 61.57396076343678
[20200116T190328] Epock 16 / 1000 loss: 62.08176247867537
[20200116T190337] Epock 

[20200116T192226] Epock 141 / 1000 loss: 52.96659716853389
[20200116T192236] Epock 142 / 1000 loss: 51.41338310712649
[20200116T192245] Epock 143 / 1000 loss: 48.59644176341869
[20200116T192254] Epock 144 / 1000 loss: 47.499957473189745
[20200116T192303] Epock 145 / 1000 loss: 47.68413461284873
[20200116T192312] Epock 146 / 1000 loss: 49.526657975750204
[20200116T192322] Epock 147 / 1000 loss: 49.54823190194589
[20200116T192331] Epock 148 / 1000 loss: 48.80183971075364
[20200116T192340] Epock 149 / 1000 loss: 47.92830066916383
[20200116T192349] Epock 150 / 1000 loss: 46.87087026054476
[20200116T192359] Epock 151 / 1000 loss: 46.85523527639884
[20200116T192408] Epock 152 / 1000 loss: 45.886754659958825
[20200116T192417] Epock 153 / 1000 loss: 45.24894848576299
[20200116T192426] Epock 154 / 1000 loss: 44.65244406240958
[20200116T192436] Epock 155 / 1000 loss: 44.636208027969175
[20200116T192445] Epock 156 / 1000 loss: 44.55177099910783
[20200116T192454] Epock 157 / 1000 loss: 44.02126856

[20200116T194337] Epock 280 / 1000 loss: 39.45002668875235
[20200116T194347] Epock 281 / 1000 loss: 37.915724224514435
[20200116T194356] Epock 282 / 1000 loss: 39.185834766905984
[20200116T194405] Epock 283 / 1000 loss: 38.90020521187488
[20200116T194414] Epock 284 / 1000 loss: 39.39622365692515
[20200116T194424] Epock 285 / 1000 loss: 37.95409261444468
[20200116T194433] Epock 286 / 1000 loss: 37.58830645054947
[20200116T194442] Epock 287 / 1000 loss: 38.719272095480086
[20200116T194451] Epock 288 / 1000 loss: 40.033114233134704
[20200116T194501] Epock 289 / 1000 loss: 37.54919662004636
[20200116T194510] Epock 290 / 1000 loss: 38.01658112325786
[20200116T194519] Epock 291 / 1000 loss: 38.759935638050976
[20200116T194528] Epock 292 / 1000 loss: 38.61010619740427
[20200116T194537] Epock 293 / 1000 loss: 38.175812992048854
[20200116T194546] Epock 294 / 1000 loss: 37.65456277352792
[20200116T194556] Epock 295 / 1000 loss: 37.022842360131534
[20200116T194605] Epock 296 / 1000 loss: 38.47664

In [ ]:
torch.save(model.state_dict(), f'checkpoint.pt.{loss}')